
# NetworkX Basics: Build, Analyze, and Visualize a Graph

This notebook walks through a compact, end‑to‑end example with [NetworkX](https://networkx.org/):

- Create a graph with nodes, edges, and attributes
- Explore basics (counts, degrees, density)
- Run common analyses (paths, components, centrality, communities)
- Visualize the graph with Matplotlib
- Save & load the graph in a standard format (GraphML)

> Tip: You can freely adapt this to your own datasets. Look for the cells marked **🔁 Customize**.


## Setup

In [ ]:

# If NetworkX / Matplotlib are not installed in your environment, uncomment the next line:
# !pip install networkx matplotlib

import networkx as nx
import matplotlib.pyplot as plt

print("NetworkX version:", nx.__version__)


## Create a small example graph (🔁 Customize)

In [ ]:

# We'll build a tiny "friendship" network with attributes.
G = nx.Graph(name="Friendship Graph")

# Add nodes with attributes
G.add_node("Alice",  age=29, city="Madrid")
G.add_node("Bob",    age=31, city="Madrid")
G.add_node("Chloe",  age=28, city="Valencia")
G.add_node("Diego",  age=35, city="Sevilla")
G.add_node("Emma",   age=33, city="Madrid")
G.add_node("Farid",  age=29, city="Bilbao")

# Add edges with an optional "weight" or "since" attribute
G.add_edge("Alice", "Bob",   since=2020)
G.add_edge("Alice", "Chloe", since=2022)
G.add_edge("Bob",   "Chloe", since=2019)
G.add_edge("Bob",   "Emma",  since=2018)
G.add_edge("Chloe", "Diego", since=2021)
G.add_edge("Emma",  "Farid", since=2020)

# A disconnected edge to demonstrate components
G.add_node("Gina", age=27, city="Zaragoza")
G.add_node("Hugo", age=30, city="Zaragoza")
G.add_edge("Gina", "Hugo", since=2024)

print(nx.info(G))
print("Nodes with data:", G.nodes(data=True))
print("Edges with data:", G.edges(data=True))


## Basic graph stats

In [ ]:

n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges()
density = nx.density(G)
degrees = dict(G.degree())

print(f"Nodes: {n_nodes}, Edges: {n_edges}, Density: {density:.3f}")
print("Degree by node:", degrees)
print("Is connected?:", nx.is_connected(G))
print("Connected components:", [sorted(c) for c in nx.connected_components(G)])


## Shortest paths

In [ ]:

# Shortest path between two nodes
source, target = "Alice", "Diego"
print(f"Shortest path from {source} to {target}:", nx.shortest_path(G, source, target))

# All-pairs shortest path lengths (small graphs only)
lengths = dict(nx.all_pairs_shortest_path_length(G))
print("Shortest path lengths from Alice:", lengths["Alice"])


## Centrality measures

In [ ]:

deg_cent = nx.degree_centrality(G)
bet_cent = nx.betweenness_centrality(G, normalized=True)
clo_cent = nx.closeness_centrality(G)

def top_k(d, k=3):
    return sorted(d.items(), key=lambda x: x[1], reverse=True)[:k]

print("Top degree centrality:", top_k(deg_cent))
print("Top betweenness centrality:", top_k(bet_cent))
print("Top closeness centrality:", top_k(clo_cent))


## Community detection (greedy modularity)

In [ ]:

from networkx.algorithms.community import greedy_modularity_communities

communities = list(greedy_modularity_communities(G))
for i, comm in enumerate(communities, start=1):
    print(f"Community {i}: {sorted(comm)}")

# Map each node -> community index for plotting
node2comm = {}
for i, comm in enumerate(communities):
    for node in comm:
        node2comm[node] = i


## Visualization

In [ ]:

# Position nodes with a spring layout (deterministic via seed)
pos = nx.spring_layout(G, seed=42)

# Node sizes by degree centrality (scaled)
sizes = [3000 * deg_cent[n] for n in G.nodes()]

# Draw
plt.figure(figsize=(8, 6))
nx.draw_networkx_nodes(G, pos, node_size=sizes, node_color=[node2comm[n] for n in G.nodes()])
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_labels(G, pos, font_size=10)

# Edge labels (friend since)
edge_labels = {(u, v): d.get("since", "") for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

plt.title(G.graph.get("name", "Graph"))
plt.axis("off")
plt.show()


## Save & load the graph (GraphML)

In [ ]:

# Save to GraphML (widely supported; great for interchange with tools like Gephi)
graphml_path = "friendship_graph.graphml"
nx.write_graphml(G, graphml_path)
print("Saved:", graphml_path)

# Load it back
G2 = nx.read_graphml(graphml_path)
print("Loaded graph has", G2.number_of_nodes(), "nodes and", G2.number_of_edges(), "edges.")


## Modify the graph (add/remove nodes & edges)

In [ ]:

# Add a new person and connect them
G.add_node("Iris", age=26, city="Madrid")
G.add_edge("Iris", "Alice", since=2025)

# Remove a node
G.remove_node("Hugo")  # Note: removes any incident edges

print("After modifications:")
print("Nodes:", sorted(G.nodes()))
print("Edges:", sorted(G.edges()))


## (Optional) Directed graph example + PageRank

In [ ]:

DG = nx.DiGraph(name="Follower Graph")
DG.add_edges_from([
    ("Alice", "Bob"),
    ("Bob", "Chloe"),
    ("Chloe", "Alice"),
    ("Emma", "Bob"),
    ("Farid", "Bob"),
    ("Diego", "Chloe"),
])

pr = nx.pagerank(DG, alpha=0.85)
print("PageRank:", dict(sorted(pr.items(), key=lambda x: x[1], reverse=True)))
